# <span style="color:#84b6f4;">Estudio para despejar incógnitas del enfoque  1-1</span>

El último experimento realizado, que se utilizó para seleccionar la arquitectura más conveniente, trajo consigo varias respuestas pero también algunas preguntas. Este estudio pretende despejar las incógnitas y comprobar si el modelo realmente aprende lo que debe aprender.

Al haber entrenado un mismo modelo con las imágenes de tres actores diferentes, se plantea la duda de si los resultados, que parecen ser un tanto impredecibles, se deben a que el modelo "reutiliza" características de entrenamientos previos y lleva a resultados engañosos, como puede ser que intente detectar a alguno de los actores para los que ha sido entrenado previamente o similar. Para ello, sería conveniente modificar el orden de entrenamiento; el modelo será entrenado con las imágenes de Bella Ramsey, Will Smith y Pedro Pascal, en ese orden. De esa forma se podrá comprobar si existe algún patrón relacionado con el orden. Además, una vez que el modelo haya sido entrenado y evaluado para los tres actores, se volverá a hacer la predicción con los actores para los que el modelo ha sido entrenado previamente, para ver si los resultados son altos, con el objetivo de comprobar si el modelo "olvida" los entrenamientos previos.

### <span style="color:#77dd77;">Creación del modelo</span>

Al igual que en el experimento anterior, trataremos de identificar a Bella Ramsey, Will Smith y Pedro Pascal utilizando el enfoque 1-1, pero en un orden diferente para ver si se mantiene el patrón en los resultados para cada actor. Comenzamos creando los DataFrames:

In [9]:
import pandas as pd
import os
import random
import cv2
import numpy as np

# Directorios en los que se encuentran las imágenes
pedro_directory = os.getcwd() + '/resources/actorImages/pedroPascal'
bella_directory = os.getcwd() + '/resources/actorImages/bellaRamsey'
will_directory = os.getcwd() + '/resources/actorImages/willSmith'
other_actors_directory = os.getcwd() + '/resources/actorImages/otherActors'

pedro_data = []
bella_data = []
will_data = []

# Añadimos imágenes en las que aparece Pedro Pascal (e imágenes en las que no)
for image in os.listdir(pedro_directory):
    image_url = os.path.join(pedro_directory, image)
    pedro_data.append({"image": image_url, "pedro_pascal": 1})

other_actor_images = os.listdir(other_actors_directory)[:150]
for image in other_actor_images:
    image_url = os.path.join(other_actors_directory, image)
    pedro_data.append({"image": image_url, "pedro_pascal": 0})

# Añadimos imágenes en las que aparece Bella Ramsey (e imágenes en las que no)
for image in os.listdir(bella_directory):
    image_url = os.path.join(bella_directory, image)
    bella_data.append({"image": image_url, "bella_ramsey": 1})

for image in other_actor_images:
    image_url = os.path.join(other_actors_directory, image)
    bella_data.append({"image": image_url, "bella_ramsey": 0})

# Añadimos imágenes en las que aparece Will Smith (e imágenes en las que no)
will_images = os.listdir(will_directory)[:150]
for image in will_images:
    image_url = os.path.join(will_directory, image)
    will_data.append({"image": image_url, "will_smith": 1})

for image in other_actor_images:
    image_url = os.path.join(other_actors_directory, image)
    will_data.append({"image": image_url, "will_smith": 0})

# Desordenamos aleatoriamente los datos
random.shuffle(pedro_data)
random.shuffle(bella_data)
random.shuffle(will_data)

# Creamos los DataFrames
pedro_df = pd.DataFrame(pedro_data)
bella_df = pd.DataFrame(bella_data)
will_df = pd.DataFrame(will_data)

Cargamos las imágenes de entrada (los píxeles) y las etiquetas asociadas a cada imagen:

In [10]:
pedro_images = []; pedro_labels = []
bella_images = []; bella_labels = []
will_images = []; will_labels = []

IMG_SIZE = 100

# Cargamos las imágenes del DataFrame de Pedro Pascal
for i, row in pedro_df.iterrows():
    image_url = row['image']
    image = cv2.imread(image_url)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = image.reshape(IMG_SIZE, IMG_SIZE, 1)
    pedro_images.append(image)
    pedro_labels.append(row['pedro_pascal'])

# Cargamos las imágenes del DataFrame de Bella Ramsey
for i, row in bella_df.iterrows():
    image_url = row['image']
    image = cv2.imread(image_url)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = image.reshape(IMG_SIZE, IMG_SIZE, 1)
    bella_images.append(image)
    bella_labels.append(row['bella_ramsey'])

# Cargamos las imágenes del DataFrame de Will Smith
for i, row in will_df.iterrows():
    image_url = row['image']
    image = cv2.imread(image_url)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = image.reshape(IMG_SIZE, IMG_SIZE, 1)
    will_images.append(image)
    will_labels.append(row['will_smith'])

pedro_images = np.array(pedro_images).astype(float)/255; pedro_labels = np.array(pedro_labels)
bella_images = np.array(bella_images).astype(float)/255; bella_labels = np.array(bella_labels)
will_images = np.array(will_images).astype(float)/255; will_labels = np.array(will_labels)

Aplicamos transformaciones de aumento de datos a las imágenes, para volver el conjunto de datos más diverso:

In [11]:
from keras.preprocessing.image import ImageDataGenerator

# Creamos un generador de imágenes aumentadas
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=15,
    zoom_range=[0.7, 1.4],
    horizontal_flip=True,
    vertical_flip=True
)

datagen.fit(pedro_images)
datagen.fit(bella_images)
datagen.fit(will_images)

Creamos un modelo de red neuronal convolucional (CNN) y lo entrenamos con las imágenes del DataFrame de Bella Ramsey:

In [12]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Dividimos los datos en conjuntos de entrenamiento (70%) y prueba (30%)
pedro_train_images, pedro_test_images, pedro_train_labels, pedro_test_labels = train_test_split(pedro_images, pedro_labels, test_size=0.3, random_state=42)
bella_train_images, bella_test_images, bella_train_labels, bella_test_labels = train_test_split(bella_images, bella_labels, test_size=0.3, random_state=42)
will_train_images, will_test_images, will_train_labels, will_test_labels = train_test_split(will_images, will_labels, test_size=0.3, random_state=42)

# Creamos el modelo
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(100, 100, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(250, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Configuramos el modelo para el entrenamiento
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenamos el modelo
model.fit(
    datagen.flow(bella_train_images, bella_train_labels, batch_size=32),
    epochs=60, batch_size=32,
    validation_data=(bella_test_images, bella_test_labels),
    steps_per_epoch=int(np.ceil(len(bella_train_images) / float(32))),
    validation_steps=int(np.ceil(len(bella_test_images) / float(32)))
)

Epoch 1/60
7/7 [==============================] - 2s 207ms/step - loss: 0.7085 - accuracy: 0.4857 - val_loss: 0.6913 - val_accuracy: 0.5111
Epoch 2/60
7/7 [==============================] - 1s 189ms/step - loss: 0.6873 - accuracy: 0.5524 - val_loss: 0.6891 - val_accuracy: 0.5000
Epoch 3/60
7/7 [==============================] - 1s 187ms/step - loss: 0.6775 - accuracy: 0.5714 - val_loss: 0.6917 - val_accuracy: 0.5111
Epoch 4/60
7/7 [==============================] - 1s 190ms/step - loss: 0.6537 - accuracy: 0.5667 - val_loss: 0.6603 - val_accuracy: 0.5556
Epoch 5/60
7/7 [==============================] - 1s 192ms/step - loss: 0.6431 - accuracy: 0.6619 - val_loss: 0.6473 - val_accuracy: 0.6444
Epoch 6/60
7/7 [==============================] - 1s 179ms/step - loss: 0.6191 - accuracy: 0.6714 - val_loss: 0.7138 - val_accuracy: 0.5667
Epoch 7/60
7/7 [==============================] - 1s 178ms/step - loss: 0.6356 - accuracy: 0.6381 - val_loss: 0.6410 - val_accuracy: 0.6333
Epoch 8/60
7/7 [====

Evaluamos los resultados del modelo, utilizando métricas como la precisión, el F1-score y la matriz de confusión:

In [13]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Evaluamos el modelo
bella_test_predictions = model.predict(bella_test_images)
bella_test_predictions_rounded = np.round(bella_test_predictions)

bella_accuracy = accuracy_score(bella_test_labels, bella_test_predictions_rounded)
bella_f1 = f1_score(bella_test_labels, bella_test_predictions_rounded)
bella_confusion = confusion_matrix(bella_test_labels, bella_test_predictions_rounded)

# Mostramos los resultados
print("Resultados para Bella:")
print("Precisión:", bella_accuracy)
print("F1 Score:", bella_f1)
print("Matriz de confusión:")
print(bella_confusion)

3/3 [==============================] - 0s 33ms/step
Resultados para Bella:
Precisión: 0.7444444444444445
F1 Score: 0.7160493827160493
Matriz de confusión:
[[38  6]
 [17 29]]


Entrenamos el modelo con las imágenes del DataFrame de Will Smith:

In [14]:
model.fit(
    datagen.flow(will_train_images, will_train_labels, batch_size=32),
    epochs=60, batch_size=32,
    validation_data=(will_test_images, will_test_labels),
    steps_per_epoch=int(np.ceil(len(will_train_images) / float(32))),
    validation_steps=int(np.ceil(len(will_test_images) / float(32)))
)

Epoch 1/60
7/7 [==============================] - 1s 202ms/step - loss: 0.7935 - accuracy: 0.5714 - val_loss: 0.6172 - val_accuracy: 0.6111
Epoch 2/60
7/7 [==============================] - 1s 187ms/step - loss: 0.6986 - accuracy: 0.5333 - val_loss: 0.7122 - val_accuracy: 0.5556
Epoch 3/60
7/7 [==============================] - 1s 199ms/step - loss: 0.6876 - accuracy: 0.5429 - val_loss: 0.7018 - val_accuracy: 0.5222
Epoch 4/60
7/7 [==============================] - 1s 196ms/step - loss: 0.6707 - accuracy: 0.5762 - val_loss: 0.6661 - val_accuracy: 0.6222
Epoch 5/60
7/7 [==============================] - 1s 190ms/step - loss: 0.6800 - accuracy: 0.5619 - val_loss: 0.6403 - val_accuracy: 0.6222
Epoch 6/60
7/7 [==============================] - 1s 184ms/step - loss: 0.6674 - accuracy: 0.6190 - val_loss: 0.6460 - val_accuracy: 0.6556
Epoch 7/60
7/7 [==============================] - 1s 195ms/step - loss: 0.6721 - accuracy: 0.5952 - val_loss: 0.6295 - val_accuracy: 0.6333
Epoch 8/60
7/7 [====

Evaluamos los resultados del modelo:

In [15]:
# Evaluamos el modelo
will_test_predictions = model.predict(will_test_images)
will_test_predictions_rounded = np.round(will_test_predictions)

will_accuracy = accuracy_score(will_test_labels, will_test_predictions_rounded)
will_f1 = f1_score(will_test_labels, will_test_predictions_rounded)
will_confusion = confusion_matrix(will_test_labels, will_test_predictions_rounded)

# Mostramos los resultados
print("Resultados para Will:")
print("Precisión:", will_accuracy)
print("F1 Score:", will_f1)
print("Matriz de confusión:")
print(will_confusion)

3/3 [==============================] - 0s 32ms/step
Resultados para Will:
Precisión: 0.6888888888888889
F1 Score: 0.6888888888888889
Matriz de confusión:
[[31 12]
 [16 31]]


Entrenamos el modelo con las imágenes del DataFrame de Pedro Pascal:

In [16]:
model.fit(
    datagen.flow(pedro_train_images, pedro_train_labels, batch_size=32),
    epochs=60, batch_size=32,
    validation_data=(pedro_test_images, pedro_test_labels),
    steps_per_epoch=int(np.ceil(len(pedro_train_images) / float(32))),
    validation_steps=int(np.ceil(len(pedro_test_images) / float(32)))
)

Epoch 1/60
7/7 [==============================] - 1s 191ms/step - loss: 0.6145 - accuracy: 0.6473 - val_loss: 0.6689 - val_accuracy: 0.6111
Epoch 2/60
7/7 [==============================] - 1s 197ms/step - loss: 0.6170 - accuracy: 0.6570 - val_loss: 0.6505 - val_accuracy: 0.6111
Epoch 3/60
7/7 [==============================] - 1s 188ms/step - loss: 0.6435 - accuracy: 0.6618 - val_loss: 0.6946 - val_accuracy: 0.5667
Epoch 4/60
7/7 [==============================] - 1s 183ms/step - loss: 0.6260 - accuracy: 0.6570 - val_loss: 0.6259 - val_accuracy: 0.6222
Epoch 5/60
7/7 [==============================] - 1s 184ms/step - loss: 0.6242 - accuracy: 0.6425 - val_loss: 0.6393 - val_accuracy: 0.5889
Epoch 6/60
7/7 [==============================] - 1s 183ms/step - loss: 0.6202 - accuracy: 0.6232 - val_loss: 0.6561 - val_accuracy: 0.5778
Epoch 7/60
7/7 [==============================] - 1s 184ms/step - loss: 0.6202 - accuracy: 0.6570 - val_loss: 0.6732 - val_accuracy: 0.5556
Epoch 8/60
7/7 [====

Evaluamos los resultados del modelo:

In [17]:
# Evaluamos el modelo
pedro_test_predictions = model.predict(pedro_test_images)
pedro_test_predictions_rounded = np.round(pedro_test_predictions)

pedro_accuracy = accuracy_score(pedro_test_labels, pedro_test_predictions_rounded)
pedro_f1 = f1_score(pedro_test_labels, pedro_test_predictions_rounded)
confusion_pedro = confusion_matrix(pedro_test_labels, pedro_test_predictions_rounded)

# Mostramos los resultados
print("Resultados para Pedro:")
print("Precisión:", pedro_accuracy)
print("F1 Score:", pedro_f1)
print("Matriz de confusión:")
print(confusion_pedro)

3/3 [==============================] - 0s 33ms/step
Resultados para Pedro:
Precisión: 0.6444444444444445
F1 Score: 0.673469387755102
Matriz de confusión:
[[25 16]
 [16 33]]


Evaluamos nuevamente el modelo con las imágenes de Will Smith y Pedro Pascal, para comprobar si el modelo "recuerda" el entrenamiento previo:

In [18]:
# Evaluamos nuevamente el modelo para Will Smith
will_test_predictions = model.predict(will_test_images)
will_test_predictions_rounded = np.round(will_test_predictions)

will_accuracy = accuracy_score(will_test_labels, will_test_predictions_rounded)
will_f1 = f1_score(will_test_labels, will_test_predictions_rounded)
will_confusion = confusion_matrix(will_test_labels, will_test_predictions_rounded)

print("Resultados para Will:")
print("Precisión:", will_accuracy)
print("F1 Score:", will_f1)
print("Matriz de confusión:")
print(will_confusion)

# Hacemos lo mismo con Bella Ramsey
bella_test_predictions = model.predict(bella_test_images)
bella_test_predictions_rounded = np.round(bella_test_predictions)

bella_accuracy = accuracy_score(bella_test_labels, bella_test_predictions_rounded)
bella_f1 = f1_score(bella_test_labels, bella_test_predictions_rounded)
confusion_bella = confusion_matrix(bella_test_labels, bella_test_predictions_rounded)

print("Resultados para Bella:")
print("Precisión:", bella_accuracy)
print("F1 Score:", bella_f1)
print("Matriz de confusión:")
print(confusion_bella)

3/3 [==============================] - 0s 25ms/step
Resultados para Will:
Precisión: 0.6555555555555556
F1 Score: 0.6593406593406593
Matriz de confusión:
[[29 14]
 [17 30]]
3/3 [==============================] - 0s 33ms/step
Resultados para Bella:
Precisión: 0.6444444444444445
F1 Score: 0.5897435897435898
Matriz de confusión:
[[35  9]
 [23 23]]


### <span style="color:#77dd77;">Conclusiones</span>

No parece haber un patrón claro de resultados que esté relacionado con el orden en el que se intentar detectar los actores. Todo parece indicar que Bella Ramsey, en general, tiene mejores resultados, independientemente del orden; así como Pedro Pascal tiene unos resultados peores. El motivo, puesto que se ha cambiado el orden y, por norma general, se sigue este patrón, parece ser que las imágenes de Bella son más indicadas para esta tarea.

Por otro lado, los resultados a la hora de detectar la presencia de Will y Bella, después de haber entrenado el modelo con imágenes de Pedro, son relativamente buenos. A pesar de que Bella suele tener mejores resultados que Will, en esta ocasión ha tenido un resultado ligeramente inferior. Lo más probable es que el motivo se deba a que el modelo se entrenó en primer lugar con imágenes suyas, y luego con las de Will, por lo que ha tenido más tiempo para "olvidarla".

La conclusión principal a la que llego tras ver los resultados es que no debo reentrenar un modelo con imágenes de otros actores. En lugar de comenzar con pesos aleatorios, a partir del segundo entrenamiento el modelo puede comenzar sesgado hacia el entrenamiento anterior, ya que parte de los pesos ajustados para la detección del actor anterior. El experimento me sigue llevando a la idea de que el enfoque 1-1 es el más conveniente, pero creando un modelo para cada actor, no reentrenando un mismo modelo una y otra vez, ya que los resultados pueden ser impredecibles.